In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn import svm

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# data to test and train on

flights_test = pd.read_csv('./flights_test.csv')
flights_train = pd.read_csv('./flights_train.csv')

flights_train = flights_train.sort_values(['AIRLINE', 'FLIGHT_NUMBER']).fillna(method='backfill')
flights_test = flights_test.sort_values(['AIRLINE', 'FLIGHT_NUMBER']).fillna(method='backfill')


In [ ]:
def aggregate_date_time(values):
    if(len(values) == 5):
        year, month, day, time, departure = values
        year = int(year)
        month = int(month)
        day = int(day)
        time = int(time)

        hour, minutes = int(time/100), time % 100
        if(hour == 24):
            hour = 0
        dateTime = datetime.datetime(year, month, day, hour, minutes)
        if( dateTime < departure and (departure-dateTime).total_seconds()/60 > 50):
            return dateTime + datetime.timedelta(days=1)
        return dateTime

       
    year, month, day, time = [int(x) for x in values]
    hour, minutes = int(time/100), time % 100
    if(hour == 24):
        hour = 0
    return datetime.datetime(year, month, day, hour, minutes)


def aggregate_time_in_minutes(dateTime):
    return dateTime.hour*60 + dateTime.minute

def preprocessing(x): 
    x['scheduled_departure_date_time'] = x[['YEAR', 'MONTH', 'DAY', 'SCHEDULED_DEPARTURE']].apply(lambda x: aggregate_date_time(x), axis=1)
    x['scheduled_departure_date'] = x['scheduled_departure_date_time'].apply(lambda x: x.date())
    x['scheduled_departure_timestamp'] = x['scheduled_departure_date_time'].apply(lambda x: x.timestamp())
    # x['scheduled_arrival_date_time'] = x[['YEAR', 'MONTH', 'DAY', 'SCHEDULED_ARRIVAL']].apply(lambda x: aggregate_date_time(x), axis=1)
    x['departure_date_time'] = x[['YEAR', 'MONTH', 'DAY', 'DEPARTURE_TIME', 'scheduled_departure_date_time']].apply(lambda x: aggregate_date_time(x), axis=1)
    x['departure_timestamp'] = x['departure_date_time'].apply(lambda x: x.timestamp())
    # x['scheduled_departure_minutes'] = x['scheduled_departure_date_time'].apply(lambda x: aggregate_time_in_minutes(x))
    x['initial_delay'] = (x['departure_date_time'] - x['scheduled_departure_date_time']).apply(lambda x: x.total_seconds() / 60)

    # aggregate mean of initial_delay during day and origin_airport
    delay_at_origin_features = ['ORIGIN_AIRPORT', 'scheduled_departure_date']
    groupedX = x.groupby(delay_at_origin_features).agg({'initial_delay': ['sum', 'mean']})
    groupedX.columns = ['_'.join(column) for column in groupedX.columns]
    x = x.join(groupedX, on=delay_at_origin_features)
    return x

In [ ]:
# Feature Selection
flights_train = preprocessing(flights_train)
flights_test = preprocessing(flights_test)

In [ ]:
origin_features = [
            'initial_delay', 
            'initial_delay_mean',
            'DISTANCE', 
            'DAY',
            'DAY_OF_WEEK',
            'MONTH',
            'ARRIVAL_DELAY'
            ]

arrival_delay = 'ARRIVAL_DELAY'

# using the full set for training
training_set = flights_train[origin_features].sample(30000)

# separating the variable that is to be predicted by the model from the rest
X = training_set.drop(arrival_delay, axis=1)
y = training_set[arrival_delay]

features = X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

#rms = mean_squared_error(y_actual, y_predicted, squared=False)

In [ ]:
#evaluation
#score = clf.score(X_test, y_test)
mse=mean_squared_error (y_test, y_pred)

print("R2: ", round(score, 3))
print("Mean Squared Error: ", round(mean_squared_error, 3))